In [6]:
# %matplotlib notebook
%matplotlib tk

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import multiprocessing

from ipywidgets import widgets

import skimage
from skimage import io
from skimage.viewer import ImageViewer


# matplotlib.rcParams['toolbar'] = 'None'



In [43]:
img = io.imread(f'images/cat01.jpg')
img = skimage.transform.rescale(img, 0.1)
print(img.shape)
img[:2]

(326, 493, 3)


array([[[0.08739901, 0.09721894, 0.09527416],
        [0.05959135, 0.0909639 , 0.09880703],
        [0.10876674, 0.11660987, 0.11268831],
        ...,
        [0.00782645, 0.02351272, 0.01959115],
        [0.01575816, 0.01963161, 0.01769488],
        [0.03517412, 0.03517412, 0.03517412]],

       [[0.07197957, 0.08766584, 0.09158741],
        [0.05068759, 0.08990327, 0.09774641],
        [0.13204877, 0.1398919 , 0.13597034],
        ...,
        [0.0147847 , 0.03047097, 0.0265494 ],
        [0.01480119, 0.02264433, 0.01872276],
        [0.03923984, 0.04126456, 0.0402522 ]]])

In [44]:
# viewer = ImageViewer(img)
# viewer.show()
f, axarr = plt.subplots(1,1)
axarr.axis('off')
axarr.imshow(img)

In [45]:
gray_image = img.copy()

y_, x_, _ = gray_image.shape
for x in range(x_):
    for y in range(y_):
        gray = np.mean(gray_image[y, x])
        gray_image[y, x] = gray

f, axarr = plt.subplots(1,1)
axarr.axis('off')
axarr.imshow(gray_image)

In [59]:
im = gray_image.copy()

x_max, y_max, _ = im.shape

def get_neighborhood(x_, y_, size):
    for x in range(x_ - size, x_ + size):
        for y in range(y_ - size, y_ + size):
            if x < 0 or y < 0 or x >= (x_max - 1) or y >= (y_max -1):
                continue
            yield im[x, y], x, y

def get_mean(x, y, size):
    g = 0
    count = 0
    for px, x, y in get_neighborhood(x, y, size):
        g += px[0]
        count += 1
    return g / count

def get_median(x, y, size):
    neighs = list(get_neighborhood(x, y, size))
    np.sort(neighs)
    mid = len(neighs) // 2
    if len(neighs) % 2 == 0:
        return neighs[mid] + neighs[mid + 1]
    return neighs[mid]

new_image = im.copy()
for x in range(0, x_max):
    for y in range(0, y_max):
        new_image[x, y] = get_median(x, y, size=5)
        
f, axarr = plt.subplots(1, 2)
for ax in axarr: ax.axis('off')

axarr[0].imshow(gray_image)
axarr[1].imshow(new_image)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()